# Step 2: Building our chat framework

## Get previously built model for getting intents

In [6]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

C:\Users\rregula\AppData\Local\Continuum\anaconda2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rregula\AppData\Local\Continuum\anaconda2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rregula\AppData\Local\Continuum\anaconda2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rregula\AppData\Loc

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)



We’ll build a simple state-machine to handle responses, using our intents model (from the previous step) as our classifier. [That’s how chatbots work](https://medium.freecodecamp.com/how-chat-bots-work-dfff656a35e2).

> A contextual chatbot framework is a classifier within a state-machine.

- we’ll un-pickle our model and documents as well as reload our intents file
-  Remember our chatbot framework is separate from our model build — you don’t need to rebuild your model unless the intent patterns change
- With several hundred intents and thousands of patterns the model could take several minutes to build

In [7]:
# restore all of our data structures
import pickle
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:
"""""input = keras.input(shape=[len(intent_train_x[0])]) 
intent_net = input
intent_net = layers.Dense(8)(intent_net) 
intent_net = layers.Dense(8)(intent_net) 
intent_net = layers.Dense(len(intent_train_y[0], activation='softmax')(intent_net) 
intent_model = keras.Model(inputs=[input], outputs=[output])
intent_model.load('model1') """""

'""input = keras.input(shape=[len(intent_train_x[0])]) \nintent_net = input\nintent_net = layers.Dense(8)(intent_net) \nintent_net = layers.Dense(8)(intent_net) \nintent_net = layers.Dense(len(intent_train_y[0], activation=\'softmax\')(intent_net) \nintent_model = keras.Model(inputs=[input], outputs=[output])\nintent_model.load(\'model1\') '

In [9]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Util functions

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return(np.array(bag))

In [11]:
p1 = bow("what is EV charging station", words)
print (p1)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
['Advantages', 'Charging cable', 'EV Brand', 'Price', 'Range', 'Types of EV', 'VOVO Electric', 'VOVO Offer', 'city', 'ev charging station', 'full Charge', 'goodbye', 'greeting', 'help', 'installation', 'opentoday', 'payments', 'safety', 'thanks', 'today']


In [12]:
# load our saved model
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\rregula\OneDrive - Infor\Downloads\chatbot-master\model.tflearn








Checking that our model gives the same result as in the previous step

In [13]:
def get_predicted_intent(predictions):
    return classes[np.argmax(predictions)]


assert 'ev charging station' == get_predicted_intent(model.predict([p1]))

## Response processor

In [19]:

# data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25


def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]

    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
        return_list.append((classes[r[0]], r[1]))

    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
                    # set context for this intent if necessary
#                     if 'context_set' in i:
#                         if show_details:
#                             print('context:', i['context_set'])
#                         context[userID] = i['context_set']

#                     # check if this intent is contextual and applies to this user's conversation
#                     if not 'context_filter' in i or \
#                             (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
#                         if show_details:
#                             print('tag:', i['tag'])

                        # a random response from the intent
                        #return print(random.choice(i['responses']))

            results.pop(0)

In [20]:
classify("EV charging  station")

[('ev charging station', 0.99526644)]

In [25]:
flag= True
print("Hi Welcome to VOVO Electric! I'm VOVOBot. Any queries?, COOL! let's chat!")
While (flag= True):
    user_response=input()
    user_response= user_response.lower()
    if (user_response= "bye"):
        if(user_response="thanks" or user_response= "thankyou"):\
            flag= False
            print("Welcome!")
        else:
            print(response(user_response))
    else:
        flag= False
        
        
        
        
        
        
        
    

SyntaxError: invalid syntax (<ipython-input-25-a419208d1e4a>, line 3)

In [12]:
response('EV charging station')

Hi there, how can I help?


In [13]:
response('do you take cash?')

Hello, thanks for visiting


In [14]:
response('where can we install?')

Good to see you again


In [15]:
classify("safety and maintenance")

[('thanks', 0.5058617), ('safety', 0.4914674)]

In [16]:
response('Is it safe to install a personalised charging station at Home?')

Hello, thanks for visiting


In [17]:
response('how safe is it?')

Good to see you again


In [18]:
response('VOVO?')

Hello, thanks for visiting


In [19]:
response('vovo offers?')

Hi there, how can I help?


In [20]:
response("does vovo offer Charging stations ")

Hello, thanks for visiting


In [21]:
context

{}

In [22]:
response('I\'d like to rent a moped')

Hello, thanks for visiting


In [23]:
context

{}

In [24]:
response('tomorrow would be great')

Hi there, how can I help?


haha

In [25]:
context

{}

In [26]:
context

{}

In [27]:
response('today please')

Hi there, how can I help?


In [28]:
classify('today')

[('today', 0.99481213)]

In [29]:
# clear context
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again


In [30]:
response('today', show_details=True)
classify('today')

Good to see you again


[('today', 0.99481213)]

In [31]:
response('THanks, that was fun')

Hello, thanks for visiting
